# Retailrocket EDA

First off: What files do we have?

首先，看一下我们有哪些数据。

In [ ]:
##USE IN GOOGLE COLAB For support of library without interuption... 

Not very big data.

数据不算太大。

In [1]:
import datetime

import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import numpy as np
import pandas as pd
import seaborn as sns
from lightfm import LightFM
from lightfm.evaluation import auc_score
from scipy.sparse import coo_matrix
from sklearn import preprocessing

sns.set()

ModuleNotFoundError: No module named 'seaborn'

Let'us do an Exploratory Data Analysis (EDA)

## Events

Users' click stream data

In [2]:
events = pd.read_csv('./events/events.csv')
print('Shape:', events.shape)
print('Columns', events.columns.tolist())
events.head()

Shape: (2756101, 5)
Columns ['timestamp', 'visitorid', 'event', 'itemid', 'transactionid']


,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [3]:
data = events.event.value_counts()

In [ ]:
labels = data.index
sizes = data.values
explode = (0, 0.1, 0.2)
fig, ax = plt.subplots(figsize=(6,6))
colors = ['b', 'g', 'r']

patches, texts, autotexts = ax.pie(sizes, labels=labels, explode=explode, autopct='%1.2f%%', shadow=False, startangle=90, colors=colors)

properties = fm.FontProperties()
properties.set_size('x-large')
# font size include: xx-small, x-small, small, medium, large, x-large, xx-large, larger, smaller, None

plt.setp(autotexts, fontproperties=properties)
plt.setp(texts, fontproperties=properties)

ax.axis('equal')
plt.show()

## Items
Unique id of the item

In [ ]:
items = events.itemid.value_counts()
for i in [2, 10, 50, 100, 1000]:
    print(f'Items that appear less than {i:>4} times:\
          {round((items < i).mean() * 100, 2)}%')

items.head(10)

In [ ]:
plt.figure(figsize=(16, 9))
plt.hist(items.values, bins=50, log=True)
plt.xlabel('Number of times item appeared', fontsize=16)
plt.ylabel('log(Count of displays with item)', fontsize=16)
plt.show()

## Visitors

In [ ]:
visitors = events.visitorid.value_counts()
for i in [2, 5, 10, 15]:
    print(f'Visitors that appear less than {i:>2} times:\
          {round((visitors < i).mean() * 100, 2):.02f}%')

visitors.head(10)

In [ ]:
plt.figure(figsize=(16, 9))
plt.hist(visitors.values, bins=50, log=True)
plt.xlabel('Number of times visitor appeared', fontsize=16)
plt.ylabel('log(Count of displays with visitor)', fontsize=16)
plt.show()

## Timestamp

In [ ]:
hour = lambda x: (datetime.datetime.fromtimestamp(x)-datetime.timedelta(hours=5)).hour
timestamp = events[['timestamp', 'event']].copy()
timestamp['timestamp'] = timestamp.timestamp / 1000
timestamp['hour'] = timestamp['timestamp'].apply(hour)

timestamp.head()

##The timestamp is UTC standard time, while the US 48 states have four time zones UTC-5 to UTC-8, assuming a UTC-5 time zone.


In [ ]:
plt.figure(figsize=(12,6))
timestamp.hour.hist(bins=np.linspace(-0.5, 23.5, 25), alpha=1, density=True)
plt.xlim(-0.5, 23.5)
plt.xlabel("Hour of Day")
plt.ylabel("Fraction of Events")
plt.show()

##The time of viewing, adding to cart, purchasing, etc. is basically in line with daily routine.

## Item Properties

In [ ]:
properties = pd.concat([pd.read_csv('../input/item_properties_part1.csv'), pd.read_csv('../input/item_properties_part2.csv')])
print('Shape:', properties.shape)
print('Columns', properties.columns.tolist())
properties.head()

All of them had been hashed excluding "categoryid" and "available", so most of the data doesn't make sense.

In [ ]:
properties = properties.loc[properties.property.isin(['categoryid', 'available']), :]
print('Shape:', properties.shape)
properties.head()

In [ ]:
categoryid = properties[properties.property=='categoryid'].drop_duplicates('itemid', keep='first')
available = properties[properties.property=='available']

In [ ]:
categoryid.head()

##My understanding is that the category corresponding to the product, the similar products can be recommended.

In [ ]:
categories = categoryid.value.value_counts()
categories.head(10)

In [ ]:
for i in [2, 10, 50, 100, 500, 1000, 5000]:
    print(f'Categories that appear less than {i:>4} times:\
          {round((categories < i).mean() * 100, 2)}%')

In [ ]:
plt.figure(figsize=(16, 9))
plt.hist(categories.values, bins=50, log=True)
plt.xlabel('Number of times categories appeared', fontsize=16)
plt.ylabel('log(Count of displays with category)', fontsize=16)
plt.show()

In [ ]:
item_category = categoryid[['itemid', 'value']]
item_category.columns = ['itemid', 'categoryid']
item_category.head()

In [ ]:
available.head()

##category tree

In [ ]:
category = pd.read_csv('../input/category_tree.csv').dropna()
print('Shape:', category.shape)
print('Columns', category.columns.tolist())
category.head()

In [ ]:
category_parent_dict = category.set_index('categoryid').T.to_dict('list')

pd.options.mode.chained_assignment = None
item_category['parentid'] = item_category.categoryid.apply(lambda x: int(category_parent_dict.get(int(x), [x])[0]))

item_category.head()

Here, we got the relationship of **itemid - categoryid - parentid**.

## Modeling by LightFM

In [ ]:
# Format the timestamp as a date and arrange it in chronological order. 
events = events.assign(date=pd.Series(datetime.datetime.fromtimestamp(i/1000).date() for i in events.timestamp))
events = events.sort_values('date').reset_index(drop=True)
events = events[['visitorid','itemid','event', 'date']]

In [ ]:
events.head()

In [ ]:
events.tail()



Since the calculation is time consuming, I am not going to use all the data.

In [ ]:
start_date = '2015-5-3'
end_date = '2015-5-18'
fd = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date()
events = events[(events.date >= fd(start_date)) & (events.date <= fd(end_date))]

In [ ]:
# Divide training sets and test sets
split_point = np.int(np.round(events.shape[0]*0.8))
events_train = events.iloc[0:split_point]
events_test = events.iloc[split_point::]
events_test = events_test[(events_test['visitorid'].isin(events_train['visitorid'])) & (events_test['itemid'].isin(events_train['itemid']))]

In [ ]:
id_cols=['visitorid','itemid']
trans_cat_train=dict()
trans_cat_test=dict()

for k in id_cols:
    cate_enc=preprocessing.LabelEncoder()
    trans_cat_train[k]=cate_enc.fit_transform(events_train[k].values)
    trans_cat_test[k]=cate_enc.transform(events_test[k].values)

In [ ]:
ratings = dict()

cate_enc=preprocessing.LabelEncoder()
ratings['train'] = cate_enc.fit_transform(events_train.event)
ratings['test'] = cate_enc.transform(events_test.event)

In [ ]:
n_users=len(np.unique(trans_cat_train['visitorid']))
n_items=len(np.unique(trans_cat_train['itemid']))

In [ ]:
rate_matrix = dict()
rate_matrix['train'] = coo_matrix((ratings['train'], (trans_cat_train['visitorid'], trans_cat_train['itemid'])), shape=(n_users,n_items))
rate_matrix['test'] = coo_matrix((ratings['test'], (trans_cat_test['visitorid'], trans_cat_test['itemid'])), shape=(n_users,n_items))

In [ ]:
model = LightFM(no_components=5, loss='warp')
model.fit(rate_matrix['train'], epochs=100, num_threads=8)

In [ ]:
auc_score(model, rate_matrix['train'], num_threads=8).mean()

In [ ]:
auc_score(model, rate_matrix['test'], num_threads=8).mean()